In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# !pip install monai

In [2]:
import os 
import torch
from torch import nn
from torch.utils.data import DataLoader

from monai.losses import DiceLoss
import numpy as np
import matplotlib.pyplot as plt
from src.models import UNet, UpNet 
from src.datagen import CustomImageDataset
from src.losses import dice_loss

In [3]:
import sys
drive_path = "drive/MyDrive/TFM/Pruebas/"
drive_path = drive_path if os.path.exists(drive_path) else "./"
sys.path.append(drive_path)

In [4]:
ini_channels = 3
out_channels = 3
width = 2
batch_size = 8
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

annotations_df = drive_path + "annotations_df.csv"
dataset_path = drive_path + "tiffs"
# annotations_df = pd.DataFrame([[f, None] for f in os.listdir(dataset_path)])
# annotations_df.to_csv("annotations_df.csv",  index=False)

In [5]:
unet = UpNet(ini_channels, out_channels, width).to(device)

cid = CustomImageDataset(annotations_df, dataset_path)
trainloader = DataLoader(cid, batch_size=batch_size, shuffle=True)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

lossf = DiceLoss(include_background=True, sigmoid=True, smooth_dr=1, smooth_nr=1,
                                           squared_pred=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(unet.parameters(), lr=1e-3, eps=1e-4)

In [7]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = []
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device) , labels.to(device) 
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = unet(inputs)
        loss = lossf(outputs.to(device), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss.append(loss.item())
        
        if i % 10 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {np.array(running_loss).mean() :.3f}')
            running_loss = []
    image = unet(inputs[0].unsqueeze(0)).permute(0, 2, 3, 1).squeeze().cpu().detach().numpy()
    plt.imshow(image)
    plt.show()

32
torch.Size([8, 16, 64, 64]) torch.Size([8, 8, 64, 64])


RuntimeError: Given groups=1, weight of size [8, 16, 3, 3], expected input[8, 24, 64, 64] to have 16 channels, but got 24 channels instead